# Notebook for running evaluations

Imports

In [ ]:
import sys
sys.path.append("..")
# from experiment.evaluations import evaluate_dispatching #, evaluate_llm_only, evaluate_baseline_rag, evaluate_reranker_rag, evaluate_dense_rag, evaluate_dense_reranker_rag
import nest_asyncio
nest_asyncio.apply()
import os
import numpy as np
import pandas as pd
import json
from db import pool 
from typing import List, Dict, Any
from scipy.stats import wilcoxon


ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
PROJECT_DIR = os.path.join(ROOT_DIR, "master-thesis-project")
EXPERIMENT_DIR = os.path.join(PROJECT_DIR, "experiment")
DATASET_DIR = os.path.join(EXPERIMENT_DIR, "data")

Run evaluation

In [3]:
# dispatching = await evaluate_dispatching()
# llm_only = await evaluate_llm_only()
# baseline_rag = await evaluate_baseline_rag()
# reranker_rag = await evaluate_reranker_rag()
# dense_rag = await evaluate_dense_rag()
# dense_reranker_rag = await evaluate_dense_reranker_rag()

Read evaluation results

In [2]:
# Dispatching
dispatching = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_dispatching.csv"))

# RAG
llm_only = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_llm_only.csv"))
baseline_rag = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_baseline_rag.csv"))
reranker_rag = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_reranker_rag.csv"))
dense_rag = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_dense_rag.csv"))
dense_reranker_rag = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_dense_reranker_rag.csv"))

Dispatching

In [12]:

# Assuming dispatching DataFrame has columns: 'expected', 'result', 'time'
if "expected" in dispatching.columns and "result" in dispatching.columns and "time" in dispatching.columns:
    dispatching["correct"] = dispatching["expected"] == dispatching["result"]
    accuracy = dispatching["correct"].mean()
    avg_time = dispatching["time"].mean()
    print(f"Dispatching accuracy: {accuracy:.4f}")
    print(f"Average response time: {avg_time:.4f} seconds")
else:
    print("Required columns ('expected', 'result', 'time') not found in dispatching DataFrame.")

# Filter out rows where result is "InputError" and recalculate summary
if "expected" in dispatching.columns and "result" in dispatching.columns and "time" in dispatching.columns:
    filtered_dispatching = dispatching[dispatching["result"] != "InputError"].copy()
    filtered_dispatching["correct"] = filtered_dispatching["expected"] == filtered_dispatching["result"]
    accuracy = filtered_dispatching["correct"].mean()
    avg_time = filtered_dispatching["time"].mean()
    print(f"Dispatching accuracy (excluding InputError): {accuracy:.4f}")
    print(f"Average response time (excluding InputError): {avg_time:.4f} seconds")

    # Group by 'expected' and count correct and incorrect predictions for filtered data
    filtered_summary = filtered_dispatching.groupby('expected').agg(
        total=('expected', 'count'),
        correct=('correct', 'sum'),
        errors=('correct', lambda x: (~x).sum())
    )
    filtered_summary['accuracy'] = filtered_summary['correct'] / filtered_summary['total']
    print(filtered_summary)
else:
    print("Required columns ('expected', 'result', 'time') not found in dispatching DataFrame.")

# Group by 'expected' and count correct and incorrect predictions
dispatching_summary = dispatching.groupby('expected').agg(
    total=('expected', 'count'),
    correct=('correct', 'sum'),
    errors=('correct', lambda x: (~x).sum())
)
dispatching_summary['accuracy'] = dispatching_summary['correct'] / dispatching_summary['total']
dispatching_summary


Dispatching accuracy: 0.8087
Average response time: 7.4747 seconds
Dispatching accuracy (excluding InputError): 0.8532
Average response time (excluding InputError): 7.4747 seconds
                   total  correct  errors  accuracy
expected                                           
Calculation Agent     59       59       0       1.0
Dreamplan Agent       80       32      48       0.4
Finance Agent        188      188       0       1.0


,total,correct,errors,accuracy
expected,,,,
Calculation Agent,59,59,0,1.000000
Dreamplan Agent,80,32,48,0.400000
Finance Agent,206,188,18,0.912621


RAG response times

In [8]:
jsonl_files = [
    os.path.join(DATASET_DIR, fname)
    for fname in os.listdir(DATASET_DIR)
    if fname.startswith("questions_") and fname.endswith(".jsonl")
]
avg_response_times = {}
for file in jsonl_files:
    response_times = []
    with open(file, "r") as f:
        for line in f:
            data = json.loads(line)
            if "time" in data:
                response_times.append(data["time"])
    if response_times:
        avg_response_times[os.path.basename(file)] = sum(response_times) / len(response_times)
    else:
        avg_response_times[os.path.basename(file)] = None

avg_response_times

{'questions_dense_reranker_rag.jsonl': 2.6997397374180916,
 'questions_llm_only.jsonl': 3.1922133038344893,
 'questions_dense_rag.jsonl': 2.4557241386580237,
 'questions_reranker_rag.jsonl': 3.042295024232957,
 'questions_baseline_rag.jsonl': 2.14312895524849}

LLM-only vs LLM + RAG

In [9]:
methods = {
    "LLM Only": llm_only,
    "Baseline RAG": baseline_rag,
    "Reranker RAG": reranker_rag,
    "Dense RAG": dense_rag,
    "Dense Reranker RAG": dense_reranker_rag,
}

# Compute mean and std for answer_relevancy for all methods,
# and for other columns for RAG methods (Baseline, Reranker, Dense)
columns_to_compare = ["answer_relevancy", "llm_context_precision_without_reference", "faithfulness", "nv_context_relevance"]

# Compare answer_relevancy for all methods
answer_relevancy_means = {method: df["answer_relevancy"].mean() for method, df in methods.items()}
answer_relevancy_df = pd.DataFrame.from_dict(answer_relevancy_means, orient="index", columns=["answer_relevancy_mean"])

# Compare other metrics for RAG methods only
rag_methods = {k: v for k, v in methods.items() if k != "LLM Only"}
other_metrics = ["llm_context_precision_without_reference", "faithfulness", "nv_context_relevance"]
other_metrics_means = {
    method: {metric: df[metric].mean() for metric in other_metrics}
    for method, df in rag_methods.items()
}
other_metrics_df = pd.DataFrame.from_dict(other_metrics_means, orient="index")

# Map method names to their corresponding avg response time file
method_to_file = {
    "LLM Only": "questions_llm_only.jsonl",
    "Baseline RAG": "questions_baseline_rag.jsonl",
    "Reranker RAG": "questions_reranker_rag.jsonl",
    "Dense RAG": "questions_dense_rag.jsonl",
    "Dense Reranker RAG": "questions_dense_reranker_rag.jsonl",
}


# Add avg response time to answer_relevancy_df
answer_relevancy_df["avg_response_time"] = [
    avg_response_times.get(method_to_file[m], None) for m in answer_relevancy_df.index
]

answer_relevancy_df

,answer_relevancy_mean,avg_response_time
LLM Only,0.770185,3.192213
Baseline RAG,0.857162,2.143129
Reranker RAG,0.842157,3.042295
Dense RAG,0.852620,2.455724
Dense Reranker RAG,0.814216,2.699740


RAG methods comparison

In [10]:
highlighted = other_metrics_df.copy()
for col in highlighted.columns:
    max_val = highlighted[col].max()
    highlighted[col] = highlighted[col].apply(
        lambda x: f"**{x:.4f}**" if np.isclose(x, max_val) else f"{x:.4f}"
    )

# Add and highlight answer_relevancy_mean
highlighted["answer_relevancy_mean"] = answer_relevancy_df.loc[highlighted.index, "answer_relevancy_mean"]
max_relevancy = highlighted["answer_relevancy_mean"].max()
highlighted["answer_relevancy_mean"] = highlighted["answer_relevancy_mean"].apply(
    lambda x: f"**{x:.4f}**" if np.isclose(x, max_relevancy) else f"{x:.4f}"
)

# Add and highlight avg_response_time (lowest is best)
highlighted["avg_response_time"] = [
    avg_response_times.get(method_to_file[m], None) for m in highlighted.index
]
min_time = highlighted["avg_response_time"].min()
highlighted["avg_response_time"] = highlighted["avg_response_time"].apply(
    lambda x: f"**{x:.4f}**" if np.isclose(x, min_time) else f"{x:.4f}"
)

highlighted


,llm_context_precision_without_reference,faithfulness,nv_context_relevance,answer_relevancy_mean,avg_response_time
Baseline RAG,0.7575,0.9259,0.5206,**0.8572**,**2.1431**
Reranker RAG,0.7410,0.9142,0.5121,0.8422,3.0423
Dense RAG,0.7673,**0.9308**,**0.6262**,0.8526,2.4557
Dense Reranker RAG,**0.7686**,0.9198,0.6238,0.8142,2.6997


Statistical significance (including LLM-only)

In [41]:

results = []

# LLM Only vs Baseline RAG
arr1 = methods["LLM Only"]["answer_relevancy"]
arr2 = methods["Baseline RAG"]["answer_relevancy"]
w_stat, w_p = wilcoxon(arr1, arr2)
results.append({
    "method_1": "LLM Only",
    "method_2": "Baseline RAG",
    "wilcoxon_p": w_p
})

# Baseline RAG vs Dense RAG
arr1 = methods["Baseline RAG"]["answer_relevancy"]
arr2 = methods["Dense RAG"]["answer_relevancy"]
w_stat, w_p = wilcoxon(arr1, arr2)
results.append({
    "method_1": "Baseline RAG",
    "method_2": "Dense RAG",
    "wilcoxon_p": w_p
})

pd.DataFrame(results)


,method_1,method_2,wilcoxon_p
0,LLM Only,Baseline RAG,1.088308e-07
1,Baseline RAG,Dense RAG,1.630178e-01


Statistical significance

In [34]:
faithfulness_baseline = baseline_rag["faithfulness"]
faithfulness_dense = dense_rag["faithfulness"]
mask = ~faithfulness_baseline.isna() & ~faithfulness_dense.isna()
_, faithfulness_p = wilcoxon(
    faithfulness_baseline[mask], faithfulness_dense[mask]
)
faithfulness_reranker = reranker_rag["faithfulness"]
mask_reranker = ~faithfulness_baseline.isna() & ~faithfulness_reranker.isna()
_, faithfulness_reranker_p = wilcoxon(
    faithfulness_baseline[mask_reranker], faithfulness_reranker[mask_reranker]
)
_, precision_p = wilcoxon(
    dense_rag["llm_context_precision_without_reference"],
    dense_reranker_rag["llm_context_precision_without_reference"]
)
_, relevance_p = wilcoxon(
    dense_rag["nv_context_relevance"], dense_reranker_rag["nv_context_relevance"]
)
_, relevance_dense_vs_reranker_p = wilcoxon(
    dense_rag["nv_context_relevance"], reranker_rag["nv_context_relevance"]
)


comparisons = [
    ("Dense vs Baseline", "faithfulness", faithfulness_p),
    ("Baseline vs Reranker", "faithfulness", faithfulness_reranker_p),
    ("Dense vs Dense+Reranker", "context precision", precision_p),
    ("Dense vs Dense+Reranker", "context relevance", relevance_p),
    ("Dense vs Baseline+Reranker", "context relevance", relevance_dense_vs_reranker_p),
]

comparison_df = pd.DataFrame(comparisons, columns=["comparison", "metric", "p-value"])
comparison_df

,comparison,metric,p-value
0,Dense vs Baseline,faithfulness,0.838382
1,Baseline vs Reranker,faithfulness,0.244506
2,Dense vs Dense+Reranker,context precision,0.011699
3,Dense vs Dense+Reranker,context relevance,0.625070
4,Dense vs Baseline+Reranker,context relevance,0.000612


System Quality

In [ ]:
async def get_feedbacks_with_stats() -> List[Dict[str, Any]]:
    """
    Return feedbacks aggregated per session:
    - number of feedbacks
    - average correctness, relevance, clarity, satisfaction
    - number of replies
    - average response time
    """
    query = """
        SELECT 
            f.session_id,
            COUNT(f.id) AS num_feedbacks,
            AVG(f.correctness) AS avg_correctness,
            AVG(f.relevance) AS avg_relevance,
            AVG(f.clarity) AS avg_clarity,
            AVG(f.satisfaction) AS avg_satisfaction,
            COUNT(r.id) AS num_replies,
            AVG(r.response_time) AS avg_response_time,
            MIN(f.timestamp) AS first_feedback,
            MAX(f.timestamp) AS last_feedback,
            ARRAY_AGG(f.comments) AS comments
        FROM feedbacks f
        LEFT JOIN replies r ON f.session_id = r.session_id
        GROUP BY f.session_id
        ORDER BY last_feedback DESC
    """
    async with pool.acquire() as conn:
        rows = await conn.fetch(query)

        # Make it cleaner for display in notebooks
        result = []
        for row in rows:
            result.append({
                "session_id": row["session_id"],
                "num_feedbacks": row["num_feedbacks"],
                "avg_scores": {
                    "correctness": float(row["avg_correctness"]) if row["avg_correctness"] is not None else None,
                    "relevance": float(row["avg_relevance"]) if row["avg_relevance"] is not None else None,
                    "clarity": float(row["avg_clarity"]) if row["avg_clarity"] is not None else None,
                    "satisfaction": float(row["avg_satisfaction"]) if row["avg_satisfaction"] is not None else None,
                },
                "num_replies": row["num_replies"],
                "avg_response_time": float(row["avg_response_time"]) if row["avg_response_time"] is not None else None,
                "first_feedback": row["first_feedback"],
                "last_feedback": row["last_feedback"],
                "comments": row["comments"] if row["comments"] else [],
            })
        return result
    
feedbacks = await get_feedbacks_with_stats()
feedbacks_df = pd.DataFrame(feedbacks)
start_time = pd.Timestamp("2025-08-18 13:22:00")

filtered_df = feedbacks_df[feedbacks_df["first_feedback"] >= start_time].copy()

scores_df = filtered_df["avg_scores"].apply(pd.Series)
filtered_df = pd.concat([filtered_df.drop(columns=["avg_scores"]), scores_df], axis=1).drop(columns=["session_id", "first_feedback", "last_feedback"])
filtered_df["avg_response_time"] = round(filtered_df["avg_response_time"] / 1000, 2)
filtered_df

In [45]:
# Print statistics for avg scores
stats = filtered_df[["correctness", "relevance", "clarity", "satisfaction"]].describe()
print("Average Scores Statistics:")
print(stats)

# Aggregate all comments into one list and print
all_comments = []
for comments in filtered_df["comments"]:
    all_comments.extend([c for c in comments if c and c.strip()])
    # Print only unique comments
unique_comments = list(dict.fromkeys(all_comments))

print("\nAggregated Unique Comments:\n")
for comment in unique_comments:
    print(comment, '\n')

Average Scores Statistics:
       correctness  relevance    clarity  satisfaction
count    41.000000  41.000000  41.000000     41.000000
mean      5.048780   5.365854   5.609756      4.878049
std       2.682827   2.233339   2.245863      2.293416
min       0.000000   0.000000   0.000000      0.000000
25%       3.000000   4.000000   4.000000      4.000000
50%       5.000000   5.000000   6.000000      5.000000
75%       7.000000   7.000000   7.000000      7.000000
max       9.000000   9.000000   9.000000      9.000000

Aggregated Unique Comments:

The bot responded promptly and provided clear and relevant answers to my questions about taxes and general financial topics. However, the investment recommendations seemed unrealistic, particularly the suggested monthly savings and pension depot amounts, which were far higher than typical financial planning guidelines. The explanation of the recommendation was detailed and helpful. Overall, the chat interface was easy to use, but recommendation